In [2]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [14]:
from openai import OpenAI

client = OpenAI()

assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Answer questions briefly, in a sentence or less.",
    model="gpt-3.5-turbo-1106",
)
MATH_ASSISTANT_ID = assistant.id
show_json(assistant)

{'id': 'asst_wrjqF0Wknnjtiiw3SJePNc7d',
 'created_at': 1703586476,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': []}

In [5]:
thread = client.beta.threads.create()
show_json(thread)

{'id': 'thread_lNn5RnOsMPvms08eVWlu5Vgi',
 'created_at': 1703585866,
 'metadata': {},
 'object': 'thread'}

In [6]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?",
)
show_json(message)

{'id': 'msg_Q1fWSr3etRrIQCPS1gzFSv6o',
 'assistant_id': None,
 'content': [{'text': {'annotations': [],
    'value': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'},
   'type': 'text'}],
 'created_at': 1703585875,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_lNn5RnOsMPvms08eVWlu5Vgi'}

In [7]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
show_json(run)

{'id': 'run_bY6zdIxfmjfuF2nMAFIzoqOz',
 'assistant_id': 'asst_DQjkaH2rXyt2ZcUIQWRdV9L0',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1703585901,
 'expires_at': 1703586501,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_lNn5RnOsMPvms08eVWlu5Vgi',
 'tools': []}

In [8]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [9]:
run = wait_on_run(run, thread)
show_json(run)

{'id': 'run_bY6zdIxfmjfuF2nMAFIzoqOz',
 'assistant_id': 'asst_DQjkaH2rXyt2ZcUIQWRdV9L0',
 'cancelled_at': None,
 'completed_at': 1703585905,
 'created_at': 1703585901,
 'expires_at': None,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1703585902,
 'status': 'completed',
 'thread_id': 'thread_lNn5RnOsMPvms08eVWlu5Vgi',
 'tools': []}

In [10]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

{'data': [{'id': 'msg_dDF8GLw1jZTSsNb55eBmgQCi',
   'assistant_id': 'asst_DQjkaH2rXyt2ZcUIQWRdV9L0',
   'content': [{'text': {'annotations': [],
      'value': 'Sure, the solution for `3x + 11 = 14` is `x = 1`.'},
     'type': 'text'}],
   'created_at': 1703585903,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_bY6zdIxfmjfuF2nMAFIzoqOz',
   'thread_id': 'thread_lNn5RnOsMPvms08eVWlu5Vgi'},
  {'id': 'msg_Q1fWSr3etRrIQCPS1gzFSv6o',
   'assistant_id': None,
   'content': [{'text': {'annotations': [],
      'value': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'},
     'type': 'text'}],
   'created_at': 1703585875,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'user',
   'run_id': None,
   'thread_id': 'thread_lNn5RnOsMPvms08eVWlu5Vgi'}],
 'object': 'list',
 'first_id': 'msg_dDF8GLw1jZTSsNb55eBmgQCi',
 'last_id': 'msg_Q1fWSr3etRrIQCPS1gzFSv6o',
 'has_more': False}

Quiz Assistant

In [11]:
def get_mock_response_from_user_multiple_choice():
    return "a"


def get_mock_response_from_user_free_response():
    return "I don't know."


def display_quiz(title, questions):
    print("Quiz:", title)
    print()
    responses = []

    for q in questions:
        print(q["question_text"])
        response = ""

        # If multiple choice, print options
        if q["question_type"] == "MULTIPLE_CHOICE":
            for i, choice in enumerate(q["choices"]):
                print(f"{i}. {choice}")
            response = get_mock_response_from_user_multiple_choice()

        # Otherwise, just get response
        elif q["question_type"] == "FREE_RESPONSE":
            response = get_mock_response_from_user_free_response()

        responses.append(response)
        print()

    return responses

In [12]:
responses = display_quiz(
    "Sample Quiz",
    [
        {"question_text": "What is your name?", "question_type": "FREE_RESPONSE"},
        {
            "question_text": "What is your favorite color?",
            "question_type": "MULTIPLE_CHOICE",
            "choices": ["Red", "Blue", "Green", "Yellow"],
        },
    ],
)
print("Responses:", responses)

Quiz: Sample Quiz

What is your name?

What is your favorite color?
0. Red
1. Blue
2. Green
3. Yellow

Responses: ["I don't know.", 'a']


In [13]:
function_json = {
    "name": "display_quiz",
    "description": "Displays a quiz to the student, and returns the student's response. A single quiz can have multiple questions.",
    "parameters": {
        "type": "object",
        "properties": {
            "title": {"type": "string"},
            "questions": {
                "type": "array",
                "description": "An array of questions, each with a title and potentially options (if multiple choice).",
                "items": {
                    "type": "object",
                    "properties": {
                        "question_text": {"type": "string"},
                        "question_type": {
                            "type": "string",
                            "enum": ["MULTIPLE_CHOICE", "FREE_RESPONSE"],
                        },
                        "choices": {"type": "array", "items": {"type": "string"}},
                    },
                    "required": ["question_text"],
                },
            },
        },
        "required": ["title", "questions"],
    },
}

In [24]:
assistant = client.beta.assistants.update(
    MATH_ASSISTANT_ID,
    tools=[
        {"type": "code_interpreter"},
        {"type": "retrieval"},
        {"type": "function", "function": function_json},
    ],
)
show_json(assistant)

{'id': 'asst_wrjqF0Wknnjtiiw3SJePNc7d',
 'created_at': 1703586476,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'},
  {'type': 'retrieval'},
  {'function': {'name': 'display_quiz',
    'description': "Displays a quiz to the student, and returns the student's response. A single quiz can have multiple questions.",
    'parameters': {'type': 'object',
     'properties': {'title': {'type': 'string'},
      'questions': {'type': 'array',
       'description': 'An array of questions, each with a title and potentially options (if multiple choice).',
       'items': {'type': 'object',
        'properties': {'question_text': {'type': 'string'},
         'question_type': {'type': 'string',
          'enum': ['MULTIPLE_CHOICE', 'FREE_RESPONSE']},
         'choices': {'type'

In [25]:
def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )


def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

def create_thread_and_run(user_input):
    thread = client.beta.threads.create()
    run = submit_message(MATH_ASSISTANT_ID, thread, user_input)
    return thread, run

In [36]:
thread, run = create_thread_and_run(
    "Make a math quiz with 2 questions: One open ended, one multiple choice. Then, give me feedback for the responses."
)
run = wait_on_run(run, thread)
run.status

'requires_action'

In [37]:
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)

print("Function Name:", name)
print("Function Arguments:")
arguments

Function Name: display_quiz
Function Arguments:


{'title': 'Math Quiz',
 'questions': [{'question_text': 'What is the square root of 64?',
   'question_type': 'FREE_RESPONSE'},
  {'question_text': 'What is the value of 3 x 5 + 2?',
   'question_type': 'MULTIPLE_CHOICE',
   'choices': ['15', '17', '20', '25']}]}

In [39]:
responses = display_quiz(arguments["title"], arguments["questions"])
print("Responses:", responses)

Quiz: Math Quiz

What is the square root of 64?

What is the value of 3 x 5 + 2?
0. 15
1. 17
2. 20
3. 25

Responses: ["I don't know.", 'a']


In [40]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_call.id,
            "output": json.dumps(responses),
        }
    ],
)
show_json(run)

{'id': 'run_BCeQqydZ7hXD4gHBOvobqw3V',
 'assistant_id': 'asst_wrjqF0Wknnjtiiw3SJePNc7d',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1703587270,
 'expires_at': 1703587870,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1703587271,
 'status': 'queued',
 'thread_id': 'thread_rmcojMjI2i5vYWVgDMGP5VKd',
 'tools': [{'type': 'code_interpreter'},
  {'type': 'retrieval'},
  {'function': {'name': 'display_quiz',
    'description': "Displays a quiz to the student, and returns the student's response. A single quiz can have multiple questions.",
    'parameters': {'type': 'object',
     'properties': {'title': {'type': 'string'},
      'questions': {'type': 'array',
       'description': 'An array of questions, each with a title and potentially options (if mu

In [41]:
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()


run = wait_on_run(run, thread)
pretty_print(get_response(thread))

# Messages
user: Make a math quiz with 2 questions: One open ended, one multiple choice. Then, give me feedback for the responses.
assistant: The square root of 64 is 8, and the correct value for 3 x 5 + 2 is 17.

